In [1]:
import os
import requests
import pandas as pd
import warnings
import psycopg2
import numpy as np
from datetime import datetime, date, timedelta
from get_spotify import *
from credentials import *
from db_loader import *
from sqlalchemy import create_engine

In [2]:
warnings.filterwarnings('ignore')
TOKEN = get_token()

In [3]:
dt_from = '2023-01-01'
dt_from = datetime.strptime(dt_from, "%Y-%m-%d")
dt_from

datetime.datetime(2023, 1, 1, 0, 0)

In [4]:
df_albums_releases = get_albums_releases(token=TOKEN, dt_from=dt_from)

date span in df: 2023-11-14 - 2024-04-05
date span in df: 2023-10-05 - 2023-12-07
'release_date'


In [5]:
df_albums = get_several_albums(TOKEN, df_albums_releases["id"])

iteration 0; 20 ids requested
iteration 1; 20 ids requested
iteration 2; 20 ids requested
iteration 3; 20 ids requested
iteration 4; 20 ids requested
iteration 5; 0 ids requested
'albums'


In [6]:
df_albums["spotify_id"] = df_albums["id"]
df_albums = df_albums[["spotify_id", "name", "release_date", "genres", "label"]]

In [12]:
merge_values(df_albums, "spotify_id", "albums")


    INSERT INTO albums (spotify_id,name,release_date,genres,label)
    VALUES %s
    ON CONFLICT(spotify_id)
    DO UPDATE SET
    	name = EXCLUDED.name,
	release_date = EXCLUDED.release_date,
	genres = EXCLUDED.genres,
	label = EXCLUDED.label
    
Values merged into albums


In [19]:
upd_fields = ""
for field in df_albums.columns:
    if field != "spotify_id":
        upd_fields = upd_fields + "\t" + field + f" = EXCLUDED.{field},\n"
        
query = f"""
INSERT INTO
VALUES %s
ON CONFLICT()
DO UPDATE SET
{upd_fields[:-2]}
"""
print(query)


INSERT INTO
VALUES %s
ON CONFLICT()
DO UPDATE SET
	name = EXCLUDED.name,
	release_date = EXCLUDED.release_date,
	genres = EXCLUDED.genres,
	label = EXCLUDED.label



In [12]:
upd_fields[:-2]

'name = EXCLUDED.{field},\nrelease_date = EXCLUDED.{field},\ngenres = EXCLUDED.{field},\nlabel = EXCLUDED.{field}'

In [ ]:
"""
name =  EXCLUDED.name,
release_date = EXCLUDED.release_date,
genres = EXCLUDED.genres,
label = EXCLUDED.label
"""

In [14]:
conn = psycopg2.connect( 
    database=os.getenv("PG_DATABASE"),
    user=os.getenv("PG_USERNAME"),
    password=os.getenv("PG_PASSWORD"),
    host=os.getenv("PG_HOST"),
    port=os.getenv("PG_PORT")
)

insert_values(conn=conn, df=df_albums, table="albums")

the dataframe is inserted


In [32]:
# get_several_artists.py

albums_ids = get_albums_releases(token)
df_albums = get_several_albums(token, albums_ids)
artists_ids = []
for alist in df_albums['artists_ids']:
    artists_ids = artists_ids + alist

headers = {
    "Authorization": f"Bearer {token}"
}
params = {
    "ids": ','.join(artists_ids)
}
response = requests.get(
    "https://api.spotify.com/v1/artists",
    headers=headers,
    params=params
)

df = pd.json_normalize(response.json()["artists"])
df_out = df[['id', 'name', 'genres', 'popularity']]

In [11]:
# get_several_tracks.py

albums_ids = get_albums_releases(token)
df_albums = get_several_albums(token, albums_ids)
tracks_ids = []
for tlist in df_albums['tracks_ids']:
    tracks_ids = tracks_ids + tlist

counter_from = 0
counter_to = min(49, len(tracks_ids))

df = pd.DataFrame()
for i in range(int(len(tracks_ids) / counter_to) + 1):
    # print(f'{counter_from}:{counter_to}')
    
    headers = {
        "Authorization": f"Bearer {token}"
    }
    params = {
        "ids": ','.join(tracks_ids[counter_from:counter_to])
    }
    response = requests.get(
        "https://api.spotify.com/v1/tracks",
        headers=headers,
        params=params
    )
    
    df_iter = pd.json_normalize(response.json()["tracks"])
    df = pd.concat([df, df_iter], ignore_index=True)
    
    counter_from += 50
    counter_to += min(50, len(tracks_ids) - counter_to)

df_tracks = df[["id", "name", "album.id", "duration_ms", "explicit", "popularity"]]
df_tracks["artists_ids"] = df["artists"].apply(lambda artists_data: [artist["id"] for artist in artists_data])

df_tracks.head()

,id,name,album.id,duration_ms,explicit,popularity,artists_ids
0,3eP13S8D5m2cweMEg3ZDed,Virginia Beach,4czdORdCWP9umpbhFXK2fW,251094,True,79,[3TVXtAsR1Inumwj472S9r4]
1,0Mrnt1YqVuW2bqmwu4VxDt,Amen (feat. Teezo Touchdown),4czdORdCWP9umpbhFXK2fW,141212,True,77,"[3TVXtAsR1Inumwj472S9r4, 0fGcIStdT1OpFFhOC7Wp36]"
2,2nibvvDdAQkVraYP00z2RS,Calling For You (feat. 21 Savage),4czdORdCWP9umpbhFXK2fW,285630,True,78,"[3TVXtAsR1Inumwj472S9r4, 1URnnhqYAYcrqrcwql10ft]"
3,53KaP77tkliz36LPxWGlWK,Fear Of Heights,4czdORdCWP9umpbhFXK2fW,155154,True,78,[3TVXtAsR1Inumwj472S9r4]
4,4ZT0wGKwlJRjpMUuwZVy3p,Daylight,4czdORdCWP9umpbhFXK2fW,164142,True,78,[3TVXtAsR1Inumwj472S9r4]


In [28]:
def iterate(num_tracks: int):
    counter_from = 0
    counter_to = min(50, num_tracks)

    for i in range(int(num_tracks / counter_to) + 1):
        print(f'{counter_from}:{counter_to}')
        counter_from += 50
        counter_to += min(50, num_tracks - counter_to)

iterate(len(tracks_ids))

0:50
50:100
100:150
150:165
